In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Daten einlesen und aufbereiten

Die Zeilen 'datetime' und 'date posted' sollen als datum interpretiert werden.
Zusätzlich soll die index spalte ignoriert werden.
Verwende dazu die optionen parse_dates und index_col, sodass die daten richtig gelesen werden.

In [ ]:
df = pd.read_csv('ufo-sightings/scrubbed.csv')

In [ ]:
df.info()

Schau dir nun die ersten 5 Einträge an

In [ ]:
df.head()

In [ ]:
df.head(5)

Wenn man versucht, die spalte longitude zu selektieren, bekommt man einen fehler.

In [ ]:
df.longitude

Was ist da los? Wir schauen uns mal die spaltennamen an:

In [ ]:
list(df)

Benennt die spalte longitude nun rightig um, dafür verwenden wir die funktion rename.

In [ ]:
df = df.rename(...)

Das datenformat ist noch nicht optimal. Duration (hours/min) ist unbrauchbar, weil es ganz verschiedene formate sind. Wir solten diese spalte entfernen und duration(seconds) in duration umbenennen.

In [ ]:
df = df.drop(['duration (hours/min)'], axis=1, errors='ignore') # Axis 1 bezieht sich auf die spalten
df = df.rename(...)
df

Wir lernen nun die function groupby kennen.
Sie gruppiert einträge anhand eines spaltenwertes.
Mit der funktion size können wir nun die grösse jeder Gruppe bestimmen

In [ ]:
shape_groups = df.groupby(['shape']).size()

In [ ]:
df.max()

In [ ]:
shape_groups

Wie kann man damit jetzt ein Tortendiagramm erstellen?

In [ ]:
shape_groups.plot.pie()

Lasst uns das noch ein bisschen Aufhübschen:


Versucht mindestens, die `figsize` zu verändern und einen Schatten einzufügen.
Sucht euch ausserdem einen Diagram stil aus.

In [ ]:
plt.style.use('seaborn-deep') # sucht euch einen style aus https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
shape_groups.plot.pie(figsize=(20,10), shadow=True)

In [ ]:
df.duration.hist(bins=10)

hmm das sieht komisch aus. Das diagramm geht bis $10^8$, also haben wir wahscheinlich ein paar ausreisser.

In [ ]:
max_seconds = 60 * 60 * 2 # 1 stunden in sekunden
len(df[df.duration > max_seconds])

Im Vergleich zu den 80000 datenpunkten die wir vorliegen haben, sind die 1 stunden sichtungen nur ein kleiner teil.

In [ ]:
len(df[df.duration > max_seconds]) / len(df)

Gibt es viele einträge mit 0?

In [ ]:
# finde mit len() und einer filterbedingung die Anzahl von 0-einträgen heraus.

Einträge mit 0 sind also nicht das problem. Viele Sichtungen sind also einfach nur sehr kurz.

Aufgabe: Was sind die minimal und maximalen werte von duration?

In [ ]:
# ...

Alles klar, unser einziges Problem sind also die ausreisser über einer Stunde.
Wir filtern also alles über einer Stunde raus, so kriegen wir

In [ ]:
df[df.duration < max_seconds].hist(column='duration')

Wie sind jetzt die Sichtungen geografisch verteilt? Dafür müssen wir ein paar packages installieren

In [ ]:
%conda install geopandas
%pip install descartes

In [ ]:
import geopandas
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))
# Kartendaten laden
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world

In [ ]:
# Kartendaten filtern und dann plotten
ax = world.plot(
    color='white', edgecolor='black', figsize=(30, 30))

gdf.plot(ax=ax, color='red')